# Widgets

If you are like a lot of data scientists, you are using `Jupyter` to manage and code your experiments and analyses. Instead of presenting static content generated by your notebooks, you can also make your notebooks interactive by using [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/index.html). Let's dive into widgets and see how they work.

## Interact

The `@interact` decorator may be placed around functions to create

- sliders
- checkboxes
- input text boxes
- select dropdowns

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(37)
plt.style.use('ggplot')

Here are two arguments, `mean` $\mu$ and `std` $\sigma$, that are associated with sliders. The sliders control the mean and standard deviation values that we used to sample from the $X \sim \mathcal{N}(\mu, \sigma)$. Sliders are controlled by a tuple: `(start, stop, step)`.

In [2]:
from ipywidgets import interact

@interact(mean=(-40, 40, 2), std=(1, 5, 0.5))
def f(mean=10, std=1):
    fig, ax = plt.subplots(figsize=(5, 3))
    s = pd.Series(np.random.normal(mean, std, 5000))
    _ = s.plot(kind='kde', ax=ax)
    _ = ax.set_title('Slider Demo, change distribution')

interactive(children=(IntSlider(value=10, description='mean', max=40, min=-40, step=2), FloatSlider(value=1.0,…

An argument that is associated with checkbox must have boolean value. In the example below, we toggle the color of positive bars to blue (`True`) or red (`False`).

In [3]:
@interact
def f(pos_blue=True):
    s = pd.Series([10, 15, -8, 7, -15])

    fig, ax = plt.subplots(figsize=(10, 3))

    color_map = {True: 'b', False: 'r'} if pos_blue else {True: 'r', False: 'b'}
    _ = s.plot(kind='bar', ax=ax, color=(s > 0).map(color_map))
    _ = ax.set_title('Bar plot, toggle colors')

interactive(children=(Checkbox(value=True, description='pos_blue'), Output()), _dom_classes=('widget-interact'…

A string argument will always create an input text box. In this example below, we convert each character to the integer representation in Unicode using `ord()` and plot out a line graph.

In [4]:
@interact
def f(text='one-off coder is the best!'):
    if len(text.strip()) == 0:
        text = 'one-off coder is the best!'
    
    s = pd.Series([ord(l) for l in list(text)])
    
    fig, ax = plt.subplots(figsize=(10, 3))
    _ = s.plot(kind='line', ax=ax)
    _ = ax.set_title(f'Line plot, {text}')

interactive(children=(Text(value='one-off coder is the best!', description='text'), Output()), _dom_classes=('…

Select dropdown boxes are created with a list of values. If the list is of tuples, each tuple must be of size two, where the first element is the display value and the second element is the backing value. 

In [5]:
df = pd.DataFrame(
    [('left', np.random.normal(5, 1)) for _ in range(100)] + 
    [('right', np.random.normal(10, 1)) for _ in range(100)], 
    columns=['handedness', 'score'])

@interact(handedness=[('left', 0), ('right', 1)])
def f(handedness=0):
    h = 'left' if handedness == 0 else 'right'
    s = df[df.handedness == h].score
    
    fig, ax = plt.subplots(figsize=(5, 3))

    _ = s.plot(kind='box', ax=ax)
    _ = ax.set_title(f'Box plot, {h} handedness, mean={s.mean():.2f}')

interactive(children=(Dropdown(description='handedness', options=(('left', 0), ('right', 1)), value=0), Output…

## interactive()

You can use `interactive()` to create multiple widgets. Below, we create two sliders, one select dropdown box and one checkbox.

In [6]:
from ipywidgets import interactive

def f(mean=3, std=2.5, color='red', small_sample=False):
    n = 100 if small_sample else 5000
    s = pd.Series(np.random.normal(mean, std, n))
    
    fig, ax = plt.subplots(figsize=(5, 3))

    _ = s.plot(kind='box', ax=ax, color=color)
    _ = ax.set_title(f'Box plot, mean={s.mean():.2f}')

w = interactive(f, mean=(-10, 10, 1), std=(1, 10, 0.5), color=['red', 'blue'], small_sample=False)
display(w)

interactive(children=(IntSlider(value=3, description='mean', max=10, min=-10), FloatSlider(value=2.5, descript…

## interact_manual()

Using `interact_manual()` will display a button to execute the attached function only if the button is executed.

In [7]:
from ipywidgets import interact_manual

w = interact_manual(f, mean=(-10, 10, 1), std=(1, 10, 0.5), color=['red', 'blue'], small_sample=False)
_ = display(w)

interactive(children=(IntSlider(value=3, description='mean', max=10, min=-10), FloatSlider(value=2.5, descript…

<function __main__.f(mean=3, std=2.5, color='red', small_sample=False)>

## interactive_output()

With `interactive_output()` you can associate widgets to parameters of a function. 

In [8]:
import ipywidgets as widgets

mean_slider = widgets.IntSlider(min=-10, max=10, step=1, value=3, description='mean')
std_slider = widgets.FloatSlider(min=0.01, max=10.0, step=0.01, value=1.0, description='standard deviation')
small_checkbox = widgets.Checkbox(description='small sample?')
color_dropdown = widgets.Dropdown(options=['red', 'blue'], value='red', description='color')

ui = widgets.HBox([mean_slider, std_slider, small_checkbox, color_dropdown])
out = widgets.interactive_output(f, {'mean': mean_slider, 'std': std_slider, 'color': color_dropdown, 'small_sample': small_checkbox})

display(ui, out)

Output()

## Other widgets

There are just [too many widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html). Let's look at the fun ones. Here's the `FloatProgress` widget.

In [9]:
import time

def do_it(fp):
    for i in range(1, 10):
        fp.value = i
        time.sleep(1)
        
fp = widgets.FloatProgress(
    value=0.0,
    min=0.0,
    max=10.0,
    step=1.0,
    description='progress:',
    bar_style='info',
    orientation='horizontal'
)

display(fp)
do_it(fp)

FloatProgress(value=0.0, bar_style='info', description='progress:', max=10.0)

Sometimes we need to pick dates.

In [10]:
dp = widgets.DatePicker(
    description='date:',
    disabled=False
)
display(dp)

DatePicker(value=None, description='date:')

At other times, colors.

In [11]:
cp = widgets.ColorPicker(
    concise=False,
    description='color:',
    value='red',
    disabled=False
)
display(cp)

ColorPicker(value='red', description='color:')

In [12]:
btn = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='danger',
    tooltip='Click me',
    icon='check'
)
display(btn)

Button(button_style='danger', description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

## Layouts

There's four basic layouts.

- box
- horizontal box
- vertical box
- grid box (advanced, but seems to be experimental at the moment)

In [13]:
widgets.Box([fp, dp, cp, btn])

Box(children=(FloatProgress(value=9.0, bar_style='info', description='progress:', max=10.0), DatePicker(value=…

In [14]:
widgets.HBox([fp, dp, cp, btn])

In [15]:
widgets.VBox([fp, dp, cp, btn])

You can use these layouts together.

In [16]:
row_0 = widgets.HBox([fp, dp])
row_1 = widgets.HBox([cp, btn])
col = widgets.VBox([row_0, row_1])
display(col)

[Out of the box layouts](https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html) are also available.